# Input Data Processing

## Set Up Code

In [3]:
import os
import pandas as pd
import re

# Set User-Defined Inputs
latest_year = '2024'
start_year = 2014
end_year = 2050
atb_data_folder = 'ATB_Data'
atb_data_file = f'{latest_year}_v3_Workbook.xlsx'

## NOTES
# change 'atb24' to 'atb{future_year}' [control-F]

## Electricity Prices

In [6]:
# Input Data Processing: Electricity Prices

# Input Folder: elec_prices
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: 'Retail Electricity Price Escalation Scenario'

## Wholesale Electricity Prices

In [21]:
# Input Data Processing: Wholesale Electricity Prices

# Input Folder: wholesale_electricity_prices
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: 'Wholesale Electricity Price Scenario'

## PV Prices

In [175]:
# Input Data Processing: PV Prices

# Input Folder: pv_prices
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: 'PV Price Scenario'
# Data Source: ATB_Data

#--------------------#

# Import the ATB data
atb_file_path = os.path.join(atb_data_folder, atb_data_file)
atb_data = pd.ExcelFile(atb_file_path)
print(f'Successfully read ATB data from: {atb_data_file}')

# Assign the individual tabs to separate data frames
pv_dist_com_full = atb_data.parse('Solar - PV Dist. Comm')
pv_dist_res_full = atb_data.parse('Solar - PV Dist. Res')

# Extract the relevant data
pv_dist_com_subset = pv_dist_com_full.iloc[157:284, 10:41] # Commercial
pv_dist_com_subset.columns = pv_dist_com_subset.iloc[0]
pv_dist_com_subset = pv_dist_com_subset[1:]
pv_dist_com_subset.columns = pv_dist_com_subset.columns.astype(float).fillna(0).astype(int).astype(str)
pv_dist_com_subset.columns.values[0] = 0
pv_dist_com_subset.columns.values[1] = 1

pv_dist_res_subset = pv_dist_res_full.iloc[157:284, 10:41] # Residential
pv_dist_res_subset.columns = pv_dist_res_subset.iloc[0]
pv_dist_res_subset = pv_dist_res_subset[1:]
pv_dist_res_subset.columns = pv_dist_res_subset.columns.astype(float).fillna(0).astype(int).astype(str)
pv_dist_res_subset.columns.values[0] = 0
pv_dist_res_subset.columns.values[1] = 1

#display(pv_dist_com_subset)
#display(pv_dist_res_subset)

##### Capital Costs
pv_dist_com_capex = pv_dist_com_subset.iloc[0:3, :] # Commercial
pv_dist_res_capex = pv_dist_res_subset.iloc[0:3, :] # Residential
#display(pv_dist_com_capex)
#display(pv_dist_res_capex)

##### Fixed Operations and Maintenance (O&M) Costs
pv_dist_com_fom = pv_dist_com_subset.iloc[64:67, :] # Commercial
pv_dist_res_fom = pv_dist_res_subset.iloc[64:67, :] # Residential
#display(pv_dist_com_fom)
#display(pv_dist_res_fom)

##### Variable Operations and Maintenance (O&M) Costs
pv_dist_com_vom = pv_dist_com_subset.iloc[96:99, :] # Commercial
pv_dist_res_vom = pv_dist_res_subset.iloc[96:99, :] # Residential
#display(pv_dist_com_vom)
#display(pv_dist_res_vom)

# Initialize empty data frames for the csv output files (conservative, moderate, advanced)
pv_prices_column_names = [
    'year',
    'system_capex_per_kw_res',
    'system_capex_per_kw_com',
    'system_capex_per_kw_ind',
    'system_om_per_kw_res',
    'system_om_per_kw_com',
    'system_om_per_kw_ind',
    'system_variable_om_per_kw_res',
    'system_variable_om_per_kw_com',
    'system_variable_om_per_kw_ind'
]
pv_price_atb24_conservative = pd.DataFrame(columns=pv_prices_column_names)
pv_price_atb24_moderate = pd.DataFrame(columns=pv_prices_column_names)
pv_price_atb24_advanced = pd.DataFrame(columns=pv_prices_column_names)

# Populate the data frames
years = list(range(start_year, end_year + 1))
pv_price_atb24_conservative['year'] = years
pv_price_atb24_moderate['year'] = years
pv_price_atb24_advanced['year'] = years

##### Capital Costs
pv_dist_com_capex_conservative = pv_dist_com_capex[pv_dist_com_capex.iloc[:, 1] == 'Conservative'] # conservative - Commercial and Industrial
pv_dist_com_capex_conservative.columns = pv_dist_com_capex_conservative.columns.astype(int)
pv_dist_com_capex_conservative_values = pv_dist_com_capex_conservative.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_conservative['system_capex_per_kw_com'] = pv_price_atb24_conservative['year'].apply(
    lambda y: pv_dist_com_capex_conservative_values[2022] if y < 2022 else pv_dist_com_capex_conservative_values.get(y, None)
)
pv_price_atb24_conservative['system_capex_per_kw_ind'] = pv_price_atb24_conservative['year'].apply(
    lambda y: pv_dist_com_capex_conservative_values[2022] if y < 2022 else pv_dist_com_capex_conservative_values.get(y, None)
)

pv_dist_res_capex_conservative = pv_dist_res_capex[pv_dist_res_capex.iloc[:, 1] == 'Conservative'] # conservative - Residential
pv_dist_res_capex_conservative.columns = pv_dist_res_capex_conservative.columns.astype(int)
pv_dist_res_capex_conservative_values = pv_dist_res_capex_conservative.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_conservative['system_capex_per_kw_res'] = pv_price_atb24_conservative['year'].apply(
    lambda y: pv_dist_res_capex_conservative_values[2022] if y < 2022 else pv_dist_res_capex_conservative_values.get(y, None)
)

pv_dist_com_capex_moderate = pv_dist_com_capex[pv_dist_com_capex.iloc[:, 1] == 'Moderate'] # moderate - Commercial and Industrial
pv_dist_com_capex_moderate.columns = pv_dist_com_capex_moderate.columns.astype(int)
pv_dist_com_capex_moderate_values = pv_dist_com_capex_moderate.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_moderate['system_capex_per_kw_com'] = pv_price_atb24_moderate['year'].apply(
    lambda y: pv_dist_com_capex_moderate_values[2022] if y < 2022 else pv_dist_com_capex_moderate_values.get(y, None)
)
pv_price_atb24_moderate['system_capex_per_kw_ind'] = pv_price_atb24_moderate['year'].apply(
    lambda y: pv_dist_com_capex_moderate_values[2022] if y < 2022 else pv_dist_com_capex_moderate_values.get(y, None)
)

pv_dist_res_capex_moderate = pv_dist_res_capex[pv_dist_res_capex.iloc[:, 1] == 'Moderate'] # moderate - Residential
pv_dist_res_capex_moderate.columns = pv_dist_res_capex_moderate.columns.astype(int)
pv_dist_res_capex_moderate_values = pv_dist_res_capex_moderate.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_moderate['system_capex_per_kw_res'] = pv_price_atb24_moderate['year'].apply(
    lambda y: pv_dist_res_capex_moderate_values[2022] if y < 2022 else pv_dist_res_capex_moderate_values.get(y, None)
)

pv_dist_com_capex_advanced = pv_dist_com_capex[pv_dist_com_capex.iloc[:, 1] == 'Advanced'] # advanced - Commercial and Industrial
pv_dist_com_capex_advanced.columns = pv_dist_com_capex_advanced.columns.astype(int)
pv_dist_com_capex_advanced_values = pv_dist_com_capex_advanced.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_advanced['system_capex_per_kw_com'] = pv_price_atb24_advanced['year'].apply(
    lambda y: pv_dist_com_capex_advanced_values[2022] if y < 2022 else pv_dist_com_capex_advanced_values.get(y, None)
)
pv_price_atb24_advanced['system_capex_per_kw_ind'] = pv_price_atb24_advanced['year'].apply(
    lambda y: pv_dist_com_capex_advanced_values[2022] if y < 2022 else pv_dist_com_capex_advanced_values.get(y, None)
)

pv_dist_res_capex_advanced = pv_dist_res_capex[pv_dist_res_capex.iloc[:, 1] == 'Advanced'] # advanced - Residential
pv_dist_res_capex_advanced.columns = pv_dist_res_capex_advanced.columns.astype(int)
pv_dist_res_capex_advanced_values = pv_dist_res_capex_advanced.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_advanced['system_capex_per_kw_res'] = pv_price_atb24_advanced['year'].apply(
    lambda y: pv_dist_res_capex_advanced_values[2022] if y < 2022 else pv_dist_res_capex_advanced_values.get(y, None)
)

##### Fixed Operations and Maintenance (O&M) Costs
pv_dist_com_fom_conservative = pv_dist_com_fom[pv_dist_com_fom.iloc[:, 1] == 'Conservative'] # conservative - Commercial and Industrial
pv_dist_com_fom_conservative.columns = pv_dist_com_fom_conservative.columns.astype(int)
pv_dist_com_fom_conservative_values = pv_dist_com_fom_conservative.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_conservative['system_om_per_kw_com'] = pv_price_atb24_conservative['year'].apply(
    lambda y: pv_dist_com_fom_conservative_values[2022] if y < 2022 else pv_dist_com_fom_conservative_values.get(y, None)
)
pv_price_atb24_conservative['system_om_per_kw_ind'] = pv_price_atb24_conservative['year'].apply(
    lambda y: pv_dist_com_fom_conservative_values[2022] if y < 2022 else pv_dist_com_fom_conservative_values.get(y, None)
)

pv_dist_res_fom_conservative = pv_dist_res_fom[pv_dist_res_fom.iloc[:, 1] == 'Conservative'] # conservative - Residential
pv_dist_res_fom_conservative.columns = pv_dist_res_fom_conservative.columns.astype(int)
pv_dist_res_fom_conservative_values = pv_dist_res_fom_conservative.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_conservative['system_om_per_kw_res'] = pv_price_atb24_conservative['year'].apply(
    lambda y: pv_dist_res_fom_conservative_values[2022] if y < 2022 else pv_dist_res_fom_conservative_values.get(y, None)
)

pv_dist_com_fom_moderate = pv_dist_com_fom[pv_dist_com_fom.iloc[:, 1] == 'Moderate'] # moderate - Commercial and Industrial
pv_dist_com_fom_moderate.columns = pv_dist_com_fom_moderate.columns.astype(int)
pv_dist_com_fom_moderate_values = pv_dist_com_fom_moderate.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_moderate['system_om_per_kw_com'] = pv_price_atb24_moderate['year'].apply(
    lambda y: pv_dist_com_fom_moderate_values[2022] if y < 2022 else pv_dist_com_fom_moderate_values.get(y, None)
)
pv_price_atb24_moderate['system_om_per_kw_ind'] = pv_price_atb24_moderate['year'].apply(
    lambda y: pv_dist_com_fom_moderate_values[2022] if y < 2022 else pv_dist_com_fom_moderate_values.get(y, None)
)

pv_dist_res_fom_moderate = pv_dist_res_fom[pv_dist_res_fom.iloc[:, 1] == 'Moderate'] # moderate - Residential
pv_dist_res_fom_moderate.columns = pv_dist_res_fom_moderate.columns.astype(int)
pv_dist_res_fom_moderate_values = pv_dist_res_fom_moderate.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_moderate['system_om_per_kw_res'] = pv_price_atb24_moderate['year'].apply(
    lambda y: pv_dist_res_fom_moderate_values[2022] if y < 2022 else pv_dist_res_fom_moderate_values.get(y, None)
)

pv_dist_com_fom_advanced = pv_dist_com_fom[pv_dist_com_fom.iloc[:, 1] == 'Advanced'] # advanced - Commercial and Industrial
pv_dist_com_fom_advanced.columns = pv_dist_com_fom_advanced.columns.astype(int)
pv_dist_com_fom_advanced_values = pv_dist_com_fom_advanced.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_advanced['system_om_per_kw_com'] = pv_price_atb24_advanced['year'].apply(
    lambda y: pv_dist_com_fom_advanced_values[2022] if y < 2022 else pv_dist_com_fom_advanced_values.get(y, None)
)
pv_price_atb24_advanced['system_om_per_kw_ind'] = pv_price_atb24_advanced['year'].apply(
    lambda y: pv_dist_com_fom_advanced_values[2022] if y < 2022 else pv_dist_com_fom_advanced_values.get(y, None)
)

pv_dist_res_fom_advanced = pv_dist_res_fom[pv_dist_res_fom.iloc[:, 1] == 'Advanced'] # advanced - Residential
pv_dist_res_fom_advanced.columns = pv_dist_res_fom_advanced.columns.astype(int)
pv_dist_res_fom_advanced_values = pv_dist_res_fom_advanced.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_advanced['system_om_per_kw_res'] = pv_price_atb24_advanced['year'].apply(
    lambda y: pv_dist_res_fom_advanced_values[2022] if y < 2022 else pv_dist_res_fom_advanced_values.get(y, None)
)

##### Variable Operations and Maintenance (O&M) Costs
pv_dist_com_vom_conservative = pv_dist_com_vom[pv_dist_com_vom.iloc[:, 1] == 'Conservative'] # conservative - Commercial and Industrial
pv_dist_com_vom_conservative.columns = pv_dist_com_vom_conservative.columns.astype(int)
pv_dist_com_vom_conservative_values = pv_dist_com_vom_conservative.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_conservative['system_variable_om_per_kw_com'] = pv_price_atb24_conservative['year'].apply(
    lambda y: pv_dist_com_vom_conservative_values[2022] if y < 2022 else pv_dist_com_vom_conservative_values.get(y, None)
)
pv_price_atb24_conservative['system_variable_om_per_kw_ind'] = pv_price_atb24_conservative['year'].apply(
    lambda y: pv_dist_com_vom_conservative_values[2022] if y < 2022 else pv_dist_com_vom_conservative_values.get(y, None)
)

pv_dist_res_vom_conservative = pv_dist_res_vom[pv_dist_res_vom.iloc[:, 1] == 'Conservative'] # conservative - Residential
pv_dist_res_vom_conservative.columns = pv_dist_res_vom_conservative.columns.astype(int)
pv_dist_res_vom_conservative_values = pv_dist_res_vom_conservative.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_conservative['system_variable_om_per_kw_res'] = pv_price_atb24_conservative['year'].apply(
    lambda y: pv_dist_res_vom_conservative_values[2022] if y < 2022 else pv_dist_res_vom_conservative_values.get(y, None)
)

pv_dist_com_vom_moderate = pv_dist_com_vom[pv_dist_com_vom.iloc[:, 1] == 'Moderate'] # moderate - Commercial and Industrial
pv_dist_com_vom_moderate.columns = pv_dist_com_vom_moderate.columns.astype(int)
pv_dist_com_vom_moderate_values = pv_dist_com_vom_moderate.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_moderate['system_variable_om_per_kw_com'] = pv_price_atb24_moderate['year'].apply(
    lambda y: pv_dist_com_vom_moderate_values[2022] if y < 2022 else pv_dist_com_vom_moderate_values.get(y, None)
)
pv_price_atb24_moderate['system_variable_om_per_kw_ind'] = pv_price_atb24_moderate['year'].apply(
    lambda y: pv_dist_com_vom_moderate_values[2022] if y < 2022 else pv_dist_com_vom_moderate_values.get(y, None)
)

pv_dist_res_vom_moderate = pv_dist_res_vom[pv_dist_res_vom.iloc[:, 1] == 'Moderate'] # moderate - Residential
pv_dist_res_vom_moderate.columns = pv_dist_res_vom_moderate.columns.astype(int)
pv_dist_res_vom_moderate_values = pv_dist_res_vom_moderate.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_moderate['system_variable_om_per_kw_res'] = pv_price_atb24_moderate['year'].apply(
    lambda y: pv_dist_res_vom_moderate_values[2022] if y < 2022 else pv_dist_res_vom_moderate_values.get(y, None)
)

pv_dist_com_vom_advanced = pv_dist_com_vom[pv_dist_com_vom.iloc[:, 1] == 'Advanced'] # advanced - Commercial and Industrial
pv_dist_com_vom_advanced.columns = pv_dist_com_vom_advanced.columns.astype(int)
pv_dist_com_vom_advanced_values = pv_dist_com_vom_advanced.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_advanced['system_variable_om_per_kw_com'] = pv_price_atb24_advanced['year'].apply(
    lambda y: pv_dist_com_vom_advanced_values[2022] if y < 2022 else pv_dist_com_vom_advanced_values.get(y, None)
)
pv_price_atb24_advanced['system_variable_om_per_kw_ind'] = pv_price_atb24_advanced['year'].apply(
    lambda y: pv_dist_com_vom_advanced_values[2022] if y < 2022 else pv_dist_com_vom_advanced_values.get(y, None)
)

pv_dist_res_vom_advanced = pv_dist_res_vom[pv_dist_res_vom.iloc[:, 1] == 'Advanced'] # advanced - Residential
pv_dist_res_vom_advanced.columns = pv_dist_res_vom_advanced.columns.astype(int)
pv_dist_res_vom_advanced_values = pv_dist_res_vom_advanced.loc[:, 2022:2050].squeeze().to_dict()
pv_price_atb24_advanced['system_variable_om_per_kw_res'] = pv_price_atb24_advanced['year'].apply(
    lambda y: pv_dist_res_vom_advanced_values[2022] if y < 2022 else pv_dist_res_vom_advanced_values.get(y, None)
)

#display(pv_price_atb24_conservative)
#display(pv_price_atb24_moderate)
#display(pv_price_atb24_advanced)

# Save the csv files
output_path = os.path.join(os.getcwd(), '..', 'pv_prices', 'pv_price_atb24_conservative.csv')
pv_price_atb24_conservative.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'pv_prices', 'pv_price_atb24_moderate.csv')
pv_price_atb24_moderate.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'pv_prices', 'pv_price_atb24_advanced.csv')
pv_price_atb24_advanced.to_csv(output_path, index=False)

Successfully read ATB data from: 2024_v3_Workbook.xlsx


## PV Performance

In [3]:
# Input Data Processing: PV Performance

# Input Folder: pv_tech_performance
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: 'PV Technical Performance Scenario'
# Data Source: ATB_Data

#--------------------#

# PV Panel Density [column names: pv_kw_per_sqft_res, pv_kw_per_sqft_com, pv_kw_per_sqft_ind]
# - This information is not in the ATB so the values from FY19 are kept the same

# PV Panel Degradation [column names: pv_degradation_factor_res, pv_degradation_factor_com, pv_degradation_factor_ind]
# - This information is not in the ATB spreadsheet, but it is on the ATB website in the capacity factor section of residential PV
# - https://atb.nrel.gov/electricity/2024/residential_pv
# - This information was manually updated for the three technology scenarios (conservative, moderate, advanced)
# - Conservative: 0.7% degradation from 2014-2050
# - Moderate: 0.7% degradation from 2014-2019, linear decrease from 0.7% to 0.5% degradation from 2020-2034, 0.5% degradation from 2035-2050
# - Moderate: 0.7% degradation from 2014-2019, linear decrease from 0.7% to 0.2% degradation from 2020-2034, 0.2% degradation from 2035-2050

# Output files
# - pv_tech_performance_atb24_conservative.csv
# - pv_tech_performance_atb24_moderate.csv
# - pv_tech_performance_atb24_advanced.csv

## Battery Prices

In [251]:
# Input Data Processing: Battery Prices

# Input Folder: batt_prices
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: 'Storage Cost Scenario'
# Data Source: ATB_Data

#--------------------#

# Import the ATB data
atb_file_path = os.path.join(atb_data_folder, atb_data_file)
atb_data = pd.ExcelFile(atb_file_path)
print(f'Successfully read ATB data from: {atb_data_file}')

# Assign the individual tabs to separate data frames
batt_com_full = atb_data.parse('Commercial Battery Storage')
batt_res_full = atb_data.parse('Residential Battery Storage')

# Extract the relevant data
batt_com_subset = batt_com_full.iloc[16:168, 4:66] # Commercial
batt_com_subset.columns = batt_com_subset.iloc[0]
batt_com_subset = batt_com_subset[1:]
batt_com_subset.columns.values[0] = 0
batt_com_subset.columns = batt_com_subset.columns.astype(float).fillna(0).astype(int).astype(str)

batt_res_subset = batt_res_full.iloc[17:113, 4:66] # Residential
batt_res_subset.columns = batt_res_subset.iloc[0]
batt_res_subset = batt_res_subset[1:]
batt_res_subset.columns.values[0] = 0
batt_res_subset.columns = batt_res_subset.columns.astype(float).fillna(0).astype(int).astype(str)

#display(batt_com_subset)
#display(batt_res_subset)

##### Capital Costs
batt_com_capex_kwh = batt_com_subset.iloc[0:3, 0:30] # Commercial - kWh
batt_com_capex_kw = batt_com_subset.iloc[6:9, 0:30] # Commercial - kW
batt_res_capex_kwh = batt_res_subset.iloc[0:3, 0:30] # Residential - kWh
batt_res_capex_kw = batt_res_subset.iloc[6:9, 0:30] # Residential - kW
#display(batt_com_capex_kwh)
#display(batt_com_capex_kw)
#display(batt_res_capex_kwh)
#display(batt_res_capex_kw)

##### Operations and Maintenance (O&M) Costs
batt_com_om_kwh = batt_com_subset.iloc[135:145, 0:31] # Commercial - kWh (Assumption: 4-hour battery)
batt_com_om_kwh.columns = batt_com_om_kwh.iloc[0]
batt_com_om_kwh = batt_com_om_kwh[1:]
batt_com_om_kwh.columns.values[0] = 0
batt_com_om_kwh.columns.values[1] = 1
batt_com_om_kwh.columns = batt_com_om_kwh.columns.astype(float).fillna(0).astype(int).astype(str)
batt_com_om_kwh = batt_com_om_kwh.iloc[6:9, :]

batt_com_om_kw = batt_com_subset.iloc[118:128, 0:31] # Commercial - kW (Assumption: 4-hour battery)
batt_com_om_kw.columns = batt_com_om_kw.iloc[0]
batt_com_om_kw = batt_com_om_kw[1:]
batt_com_om_kw.columns.values[0] = 0
batt_com_om_kw.columns.values[1] = 1
batt_com_om_kw.columns = batt_com_om_kwh.columns.astype(float).fillna(0).astype(int).astype(str)
batt_com_om_kw = batt_com_om_kw.iloc[6:9, :]

batt_res_om_kwh = batt_res_subset.iloc[88:92, 0:31] # Residential - kWh (Assumption: 5 kW - 12.5 kWh battery)
batt_res_om_kwh.columns = batt_res_om_kwh.iloc[0]
batt_res_om_kwh = batt_res_om_kwh[1:]
batt_res_om_kwh.columns.values[0] = 0
batt_res_om_kwh.columns.values[1] = 1
batt_res_om_kwh.columns = batt_res_om_kwh.columns.astype(float).fillna(0).astype(int).astype(str)

batt_res_om_kw = batt_res_subset.iloc[80:84, 0:31] # Residential - kW (Assumption: 5 kW - 12.5 kWh battery)
batt_res_om_kw.columns = batt_res_om_kw.iloc[0]
batt_res_om_kw = batt_res_om_kw[1:]
batt_res_om_kw.columns.values[0] = 0
batt_res_om_kw.columns.values[1] = 1
batt_res_om_kw.columns = batt_res_om_kw.columns.astype(float).fillna(0).astype(int).astype(str)

#display(batt_com_om_kwh)
#display(batt_com_om_kw)
#display(batt_res_om_kwh)
#display(batt_res_om_kw)

##### Linear Constant
batt_com_constant = batt_com_subset.iloc[0:3, 31:62] # Commercial
batt_res_constant = batt_res_subset.iloc[0:3, 32:62] # Residential
#display(batt_com_constant)
#display(batt_res_constant)

# Initialize empty data frames for the csv output files (conservative, moderate, advanced)
batt_prices_column_names = [
    'year',
    'batt_capex_per_kwh_res',
    'batt_capex_per_kw_res',
    'batt_capex_per_kwh_nonres',
    'batt_capex_per_kw_nonres',
    'linear_constant_res',
    'linear_constant_nonres',
    'batt_om_per_kw_res',
    'batt_om_per_kwh_res',
    'batt_om_per_kw_nonres',
    'batt_om_per_kwh_nonres',
    'batt_replace_frac_kw',
    'batt_replace_frac_kwh'
]
batt_price_atb24_conservative = pd.DataFrame(columns=batt_prices_column_names)
batt_price_atb24_moderate = pd.DataFrame(columns=batt_prices_column_names)
batt_price_atb24_advanced = pd.DataFrame(columns=batt_prices_column_names)

# Populate the data frames
years = list(range(start_year, end_year + 1))
batt_price_atb24_conservative['year'] = years
batt_price_atb24_moderate['year'] = years
batt_price_atb24_advanced['year'] = years
batt_price_atb24_conservative['batt_replace_frac_kw'] = 0.174
batt_price_atb24_moderate['batt_replace_frac_kw'] = 0.174
batt_price_atb24_advanced['batt_replace_frac_kw'] = 0.174
batt_price_atb24_conservative['batt_replace_frac_kwh'] = 1
batt_price_atb24_moderate['batt_replace_frac_kwh'] = 1
batt_price_atb24_advanced['batt_replace_frac_kwh'] = 1

##### Capital Costs
batt_com_capex_kwh_conservative = batt_com_capex_kwh[batt_com_capex_kwh.iloc[:, 0] == 'Conservative'] # conservative - Nonresidential
batt_com_capex_kwh_conservative.columns = batt_com_capex_kwh_conservative.columns.astype(int)
batt_com_capex_kwh_conservative_values = batt_com_capex_kwh_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_conservative['batt_capex_per_kwh_nonres'] = batt_price_atb24_conservative['year'].apply(
    lambda y: batt_com_capex_kwh_conservative_values[2022] if y < 2022 else batt_com_capex_kwh_conservative_values.get(y, None)
)
batt_com_capex_kw_conservative = batt_com_capex_kw[batt_com_capex_kw.iloc[:, 0] == 'Conservative'] # conservative - Nonresidential
batt_com_capex_kw_conservative.columns = batt_com_capex_kw_conservative.columns.astype(int)
batt_com_capex_kw_conservative_values = batt_com_capex_kw_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_conservative['batt_capex_per_kw_nonres'] = batt_price_atb24_conservative['year'].apply(
    lambda y: batt_com_capex_kw_conservative_values[2022] if y < 2022 else batt_com_capex_kw_conservative_values.get(y, None)
)
batt_res_capex_kwh_conservative = batt_res_capex_kwh[batt_res_capex_kwh.iloc[:, 0] == 'Conservative'] # conservative - Residential
batt_res_capex_kwh_conservative.columns = batt_res_capex_kwh_conservative.columns.astype(int)
batt_res_capex_kwh_conservative_values = batt_res_capex_kwh_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_conservative['batt_capex_per_kwh_res'] = batt_price_atb24_conservative['year'].apply(
    lambda y: batt_res_capex_kwh_conservative_values[2022] if y < 2022 else batt_res_capex_kwh_conservative_values.get(y, None)
)
batt_res_capex_kw_conservative = batt_res_capex_kw[batt_res_capex_kw.iloc[:, 0] == 'Conservative'] # conservative - Residential
batt_res_capex_kw_conservative.columns = batt_res_capex_kw_conservative.columns.astype(int)
batt_res_capex_kw_conservative_values = batt_res_capex_kw_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_conservative['batt_capex_per_kw_res'] = batt_price_atb24_conservative['year'].apply(
    lambda y: batt_res_capex_kw_conservative_values[2022] if y < 2022 else batt_res_capex_kw_conservative_values.get(y, None)
)

batt_com_capex_kwh_moderate = batt_com_capex_kwh[batt_com_capex_kwh.iloc[:, 0] == 'Moderate'] # moderate - Nonresidential
batt_com_capex_kwh_moderate.columns = batt_com_capex_kwh_moderate.columns.astype(int)
batt_com_capex_kwh_moderate_values = batt_com_capex_kwh_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_moderate['batt_capex_per_kwh_nonres'] = batt_price_atb24_moderate['year'].apply(
    lambda y: batt_com_capex_kwh_moderate_values[2022] if y < 2022 else batt_com_capex_kwh_moderate_values.get(y, None)
)
batt_com_capex_kw_moderate = batt_com_capex_kw[batt_com_capex_kw.iloc[:, 0] == 'Moderate'] # moderate - Nonresidential
batt_com_capex_kw_moderate.columns = batt_com_capex_kw_moderate.columns.astype(int)
batt_com_capex_kw_moderate_values = batt_com_capex_kw_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_moderate['batt_capex_per_kw_nonres'] = batt_price_atb24_moderate['year'].apply(
    lambda y: batt_com_capex_kw_moderate_values[2022] if y < 2022 else batt_com_capex_kw_moderate_values.get(y, None)
)
batt_res_capex_kwh_moderate = batt_res_capex_kwh[batt_res_capex_kwh.iloc[:, 0] == 'Moderate'] # moderate - Residential
batt_res_capex_kwh_moderate.columns = batt_res_capex_kwh_moderate.columns.astype(int)
batt_res_capex_kwh_moderate_values = batt_res_capex_kwh_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_moderate['batt_capex_per_kwh_res'] = batt_price_atb24_moderate['year'].apply(
    lambda y: batt_res_capex_kwh_moderate_values[2022] if y < 2022 else batt_res_capex_kwh_moderate_values.get(y, None)
)
batt_res_capex_kw_moderate = batt_res_capex_kw[batt_res_capex_kw.iloc[:, 0] == 'Moderate'] # moderate - Residential
batt_res_capex_kw_moderate.columns = batt_res_capex_kw_moderate.columns.astype(int)
batt_res_capex_kw_moderate_values = batt_res_capex_kw_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_moderate['batt_capex_per_kw_res'] = batt_price_atb24_moderate['year'].apply(
    lambda y: batt_res_capex_kw_moderate_values[2022] if y < 2022 else batt_res_capex_kw_moderate_values.get(y, None)
)

batt_com_capex_kwh_advanced = batt_com_capex_kwh[batt_com_capex_kwh.iloc[:, 0] == 'Advanced'] # advanced - Nonresidential
batt_com_capex_kwh_advanced.columns = batt_com_capex_kwh_advanced.columns.astype(int)
batt_com_capex_kwh_advanced_values = batt_com_capex_kwh_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_advanced['batt_capex_per_kwh_nonres'] = batt_price_atb24_advanced['year'].apply(
    lambda y: batt_com_capex_kwh_advanced_values[2022] if y < 2022 else batt_com_capex_kwh_advanced_values.get(y, None)
)
batt_com_capex_kw_advanced = batt_com_capex_kw[batt_com_capex_kw.iloc[:, 0] == 'Advanced'] # advanced - Nonresidential
batt_com_capex_kw_advanced.columns = batt_com_capex_kw_advanced.columns.astype(int)
batt_com_capex_kw_advanced_values = batt_com_capex_kw_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_advanced['batt_capex_per_kw_nonres'] = batt_price_atb24_advanced['year'].apply(
    lambda y: batt_com_capex_kw_advanced_values[2022] if y < 2022 else batt_com_capex_kw_advanced_values.get(y, None)
)
batt_res_capex_kwh_advanced = batt_res_capex_kwh[batt_res_capex_kwh.iloc[:, 0] == 'Advanced'] # advanced - Residential
batt_res_capex_kwh_advanced.columns = batt_res_capex_kwh_advanced.columns.astype(int)
batt_res_capex_kwh_advanced_values = batt_res_capex_kwh_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_advanced['batt_capex_per_kwh_res'] = batt_price_atb24_advanced['year'].apply(
    lambda y: batt_res_capex_kwh_advanced_values[2022] if y < 2022 else batt_res_capex_kwh_advanced_values.get(y, None)
)
batt_res_capex_kw_advanced = batt_res_capex_kw[batt_res_capex_kw.iloc[:, 0] == 'Advanced'] # advanced - Residential
batt_res_capex_kw_advanced.columns = batt_res_capex_kw_advanced.columns.astype(int)
batt_res_capex_kw_advanced_values = batt_res_capex_kw_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_advanced['batt_capex_per_kw_res'] = batt_price_atb24_advanced['year'].apply(
    lambda y: batt_res_capex_kw_advanced_values[2022] if y < 2022 else batt_res_capex_kw_advanced_values.get(y, None)
)

##### Operations and Maintenance (O&M) Costs
batt_com_om_kwh_conservative = batt_com_om_kwh[batt_com_om_kwh.iloc[:, 1] == 'Conservative'] # conservative - Nonresidential
batt_com_om_kwh_conservative.columns = batt_com_om_kwh_conservative.columns.astype(int)
batt_com_om_kwh_conservative_values = batt_com_om_kwh_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_conservative['batt_om_per_kwh_nonres'] = batt_price_atb24_conservative['year'].apply(
    lambda y: batt_com_om_kwh_conservative_values[2022] if y < 2022 else batt_com_om_kwh_conservative_values.get(y, None)
)
batt_com_om_kw_conservative = batt_com_om_kw[batt_com_om_kw.iloc[:, 1] == 'Conservative'] # conservative - Nonresidential
batt_com_om_kw_conservative.columns = batt_com_om_kw_conservative.columns.astype(int)
batt_com_om_kw_conservative_values = batt_com_om_kw_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_conservative['batt_om_per_kw_nonres'] = batt_price_atb24_conservative['year'].apply(
    lambda y: batt_com_om_kw_conservative_values[2022] if y < 2022 else batt_com_om_kw_conservative_values.get(y, None)
)
batt_res_om_kwh_conservative = batt_res_om_kwh[batt_res_om_kwh.iloc[:, 1] == 'Conservative'] # conservative - Residential
batt_res_om_kwh_conservative.columns = batt_res_om_kwh_conservative.columns.astype(int)
batt_res_om_kwh_conservative_values = batt_res_om_kwh_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_conservative['batt_om_per_kwh_res'] = batt_price_atb24_conservative['year'].apply(
    lambda y: batt_res_om_kwh_conservative_values[2022] if y < 2022 else batt_res_om_kwh_conservative_values.get(y, None)
)
batt_res_om_kw_conservative = batt_res_om_kw[batt_res_om_kw.iloc[:, 1] == 'Conservative'] # conservative - Residential
batt_res_om_kw_conservative.columns = batt_res_om_kw_conservative.columns.astype(int)
batt_res_om_kw_conservative_values = batt_res_om_kw_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_conservative['batt_om_per_kw_res'] = batt_price_atb24_conservative['year'].apply(
    lambda y: batt_res_om_kw_conservative_values[2022] if y < 2022 else batt_res_om_kw_conservative_values.get(y, None)
)

batt_com_om_kwh_moderate = batt_com_om_kwh[batt_com_om_kwh.iloc[:, 1] == 'Moderate'] # moderate - Nonresidential
batt_com_om_kwh_moderate.columns = batt_com_om_kwh_moderate.columns.astype(int)
batt_com_om_kwh_moderate_values = batt_com_om_kwh_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_moderate['batt_om_per_kwh_nonres'] = batt_price_atb24_moderate['year'].apply(
    lambda y: batt_com_om_kwh_moderate_values[2022] if y < 2022 else batt_com_om_kwh_moderate_values.get(y, None)
)
batt_com_om_kw_moderate = batt_com_om_kw[batt_com_om_kw.iloc[:, 1] == 'Moderate'] # moderate - Nonresidential
batt_com_om_kw_moderate.columns = batt_com_om_kw_moderate.columns.astype(int)
batt_com_om_kw_moderate_values = batt_com_om_kw_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_moderate['batt_om_per_kw_nonres'] = batt_price_atb24_moderate['year'].apply(
    lambda y: batt_com_om_kw_moderate_values[2022] if y < 2022 else batt_com_om_kw_moderate_values.get(y, None)
)
batt_res_om_kwh_moderate = batt_res_om_kwh[batt_res_om_kwh.iloc[:, 1] == 'Moderate'] # moderate - Residential
batt_res_om_kwh_moderate.columns = batt_res_om_kwh_moderate.columns.astype(int)
batt_res_om_kwh_moderate_values = batt_res_om_kwh_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_moderate['batt_om_per_kwh_res'] = batt_price_atb24_moderate['year'].apply(
    lambda y: batt_res_om_kwh_moderate_values[2022] if y < 2022 else batt_res_om_kwh_moderate_values.get(y, None)
)
batt_res_om_kw_moderate = batt_res_om_kw[batt_res_om_kw.iloc[:, 1] == 'Moderate'] # moderate - Residential
batt_res_om_kw_moderate.columns = batt_res_om_kw_moderate.columns.astype(int)
batt_res_om_kw_moderate_values = batt_res_om_kw_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_moderate['batt_om_per_kw_res'] = batt_price_atb24_moderate['year'].apply(
    lambda y: batt_res_om_kw_moderate_values[2022] if y < 2022 else batt_res_om_kw_moderate_values.get(y, None)
)

batt_com_om_kwh_advanced = batt_com_om_kwh[batt_com_om_kwh.iloc[:, 1] == 'Advanced'] # advanced - Nonresidential
batt_com_om_kwh_advanced.columns = batt_com_om_kwh_advanced.columns.astype(int)
batt_com_om_kwh_advanced_values = batt_com_om_kwh_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_advanced['batt_om_per_kwh_nonres'] = batt_price_atb24_advanced['year'].apply(
    lambda y: batt_com_om_kwh_advanced_values[2022] if y < 2022 else batt_com_om_kwh_advanced_values.get(y, None)
)
batt_com_om_kw_advanced = batt_com_om_kw[batt_com_om_kw.iloc[:, 1] == 'Advanced'] # advanced - Nonresidential
batt_com_om_kw_advanced.columns = batt_com_om_kw_advanced.columns.astype(int)
batt_com_om_kw_advanced_values = batt_com_om_kw_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_advanced['batt_om_per_kw_nonres'] = batt_price_atb24_advanced['year'].apply(
    lambda y: batt_com_om_kw_advanced_values[2022] if y < 2022 else batt_com_om_kw_advanced_values.get(y, None)
)
batt_res_om_kwh_advanced = batt_res_om_kwh[batt_res_om_kwh.iloc[:, 1] == 'Advanced'] # advanced - Residential
batt_res_om_kwh_advanced.columns = batt_res_om_kwh_advanced.columns.astype(int)
batt_res_om_kwh_advanced_values = batt_res_om_kwh_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_advanced['batt_om_per_kwh_res'] = batt_price_atb24_advanced['year'].apply(
    lambda y: batt_res_om_kwh_advanced_values[2022] if y < 2022 else batt_res_om_kwh_advanced_values.get(y, None)
)
batt_res_om_kw_advanced = batt_res_om_kw[batt_res_om_kw.iloc[:, 1] == 'Advanced'] # advanced - Residential
batt_res_om_kw_advanced.columns = batt_res_om_kw_advanced.columns.astype(int)
batt_res_om_kw_advanced_values = batt_res_om_kw_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_advanced['batt_om_per_kw_res'] = batt_price_atb24_advanced['year'].apply(
    lambda y: batt_res_om_kw_advanced_values[2022] if y < 2022 else batt_res_om_kw_advanced_values.get(y, None)
)

##### Linear Constant
batt_com_constant_conservative = batt_com_constant[batt_com_constant.iloc[:, 0] == 'Conservative'] # conservative - Nonresidential
batt_com_constant_conservative.columns = batt_com_constant_conservative.columns.astype(int)
batt_com_constant_conservative_values = batt_com_constant_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_conservative['linear_constant_nonres'] = batt_price_atb24_conservative['year'].apply(
    lambda y: batt_com_constant_conservative_values[2022] if y < 2022 else batt_com_constant_conservative_values.get(y, None)
)
batt_res_constant_conservative = batt_res_constant[batt_res_constant.iloc[:, 0] == 'Conservative'] # conservative - Residential
batt_res_constant_conservative.columns = batt_res_constant_conservative.columns.astype(int)
batt_res_constant_conservative_values = batt_res_constant_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_conservative['linear_constant_res'] = batt_price_atb24_conservative['year'].apply(
    lambda y: batt_res_constant_conservative_values[2022] if y < 2022 else batt_res_constant_conservative_values.get(y, None)
)

batt_com_constant_moderate = batt_com_constant[batt_com_constant.iloc[:, 0] == 'Moderate'] # moderate - Nonresidential
batt_com_constant_moderate.columns = batt_com_constant_moderate.columns.astype(int)
batt_com_constant_moderate_values = batt_com_constant_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_moderate['linear_constant_nonres'] = batt_price_atb24_moderate['year'].apply(
    lambda y: batt_com_constant_moderate_values[2022] if y < 2022 else batt_com_constant_moderate_values.get(y, None)
)
batt_res_constant_moderate = batt_res_constant[batt_res_constant.iloc[:, 0] == 'Moderate'] # moderate - Residential
batt_res_constant_moderate.columns = batt_res_constant_moderate.columns.astype(int)
batt_res_constant_moderate_values = batt_res_constant_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_moderate['linear_constant_res'] = batt_price_atb24_moderate['year'].apply(
    lambda y: batt_res_constant_moderate_values[2022] if y < 2022 else batt_res_constant_moderate_values.get(y, None)
)

batt_com_constant_advanced = batt_com_constant[batt_com_constant.iloc[:, 0] == 'Advanced'] # advanced - Nonresidential
batt_com_constant_advanced.columns = batt_com_constant_advanced.columns.astype(int)
batt_com_constant_advanced_values = batt_com_constant_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_advanced['linear_constant_nonres'] = batt_price_atb24_advanced['year'].apply(
    lambda y: batt_com_constant_advanced_values[2022] if y < 2022 else batt_com_constant_advanced_values.get(y, None)
)
batt_res_constant_advanced = batt_res_constant[batt_res_constant.iloc[:, 0] == 'Advanced'] # advanced - Residential
batt_res_constant_advanced.columns = batt_res_constant_advanced.columns.astype(int)
batt_res_constant_advanced_values = batt_res_constant_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_price_atb24_advanced['linear_constant_res'] = batt_price_atb24_advanced['year'].apply(
    lambda y: batt_res_constant_advanced_values[2022] if y < 2022 else batt_res_constant_advanced_values.get(y, None)
)

#display(batt_price_atb24_conservative)
#display(batt_price_atb24_moderate)
#display(batt_price_atb24_advanced)

# Save the csv files
output_path = os.path.join(os.getcwd(), '..', 'batt_prices', 'batt_price_atb24_conservative.csv')
batt_price_atb24_conservative.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'batt_prices', 'batt_price_atb24_moderate.csv')
batt_price_atb24_moderate.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'batt_prices', 'batt_price_atb24_advanced.csv')
batt_price_atb24_advanced.to_csv(output_path, index=False)

Successfully read ATB data from: 2024_v3_Workbook.xlsx


## Battery Performance

In [283]:
# Input Data Processing: Battery Performance

# Input Folder: batt_tech_performance
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: 'Storage Technical Performance Scenario'
# Data Source: ATB_Data

#--------------------#

# Import the ATB data
atb_file_path = os.path.join(atb_data_folder, atb_data_file)
atb_data = pd.ExcelFile(atb_file_path)
print(f'Successfully read ATB data from: {atb_data_file}')

# Assign the individual tabs to separate data frames
batt_com_full = atb_data.parse('Commercial Battery Storage')
batt_res_full = atb_data.parse('Residential Battery Storage')

# Extract the relevant data
##### Efficiency
batt_com_eff = batt_com_full.iloc[186:190, 4:35] # Commercial
batt_com_eff.columns = batt_com_eff.iloc[0]
batt_com_eff = batt_com_eff[1:]
batt_com_eff.columns.values[0] = 0
batt_com_eff.columns.values[1] = 1
batt_com_eff.columns = batt_com_subset.columns.astype(float).fillna(0).astype(int).astype(str)

batt_res_eff = batt_res_full.iloc[122:126, 4:35] # Residential
batt_res_eff.columns = batt_res_eff.iloc[0]
batt_res_eff = batt_res_eff[1:]
batt_res_eff.columns.values[0] = 0
batt_res_eff.columns.values[1] = 1
batt_res_eff.columns = batt_res_eff.columns.astype(float).fillna(0).astype(int).astype(str)

#display(batt_com_eff)
#display(batt_res_eff)

# Initialize empty data frames for the csv output files (conservative, moderate, advanced)
batt_tech_performance_column_names = [
    'year',
    'batt_eff_res',
    'batt_eff_com',
    'batt_eff_ind',
    'batt_lifetime_yrs_res',
    'batt_lifetime_yrs_com',
    'batt_lifetime_yrs_ind',
]
batt_tech_performance_atb24_conservative = pd.DataFrame(columns=batt_tech_performance_column_names)
batt_tech_performance_atb24_moderate = pd.DataFrame(columns=batt_tech_performance_column_names)
batt_tech_performance_atb24_advanced = pd.DataFrame(columns=batt_tech_performance_column_names)

# Populate the data frames
years = list(range(start_year, end_year + 1))
batt_tech_performance_atb24_conservative['year'] = years
batt_tech_performance_atb24_moderate['year'] = years
batt_tech_performance_atb24_advanced['year'] = years
batt_tech_performance_atb24_conservative['batt_lifetime_yrs_res'] = 10
batt_tech_performance_atb24_moderate['batt_lifetime_yrs_res'] = 10
batt_tech_performance_atb24_advanced['batt_lifetime_yrs_res'] = 10
batt_tech_performance_atb24_conservative['batt_lifetime_yrs_com'] = 10
batt_tech_performance_atb24_moderate['batt_lifetime_yrs_com'] = 10
batt_tech_performance_atb24_advanced['batt_lifetime_yrs_com'] = 10
batt_tech_performance_atb24_conservative['batt_lifetime_yrs_ind'] = 10
batt_tech_performance_atb24_moderate['batt_lifetime_yrs_ind'] = 10
batt_tech_performance_atb24_advanced['batt_lifetime_yrs_ind'] = 10

##### Efficiency
batt_com_eff_conservative = batt_com_eff[batt_com_eff.iloc[:, 1] == 'Conservative'] # conservative - Commercial and Industrial
batt_com_eff_conservative.columns = batt_com_eff_conservative.columns.astype(int)
batt_com_eff_conservative_values = batt_com_eff_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_tech_performance_atb24_conservative['batt_eff_com'] = batt_tech_performance_atb24_conservative['year'].apply(
    lambda y: batt_com_eff_conservative_values[2022] if y < 2022 else batt_com_eff_conservative_values.get(y, None)
)
batt_tech_performance_atb24_conservative['batt_eff_ind'] = batt_tech_performance_atb24_conservative['year'].apply(
    lambda y: batt_com_eff_conservative_values[2022] if y < 2022 else batt_com_eff_conservative_values.get(y, None)
)

batt_res_eff_conservative = batt_res_eff[batt_res_eff.iloc[:, 1] == 'Conservative'] # conservative - Residential
batt_res_eff_conservative.columns = batt_res_eff_conservative.columns.astype(int)
batt_res_eff_conservative_values = batt_res_eff_conservative.loc[:, 2022:2050].squeeze().to_dict()
batt_tech_performance_atb24_conservative['batt_eff_res'] = batt_tech_performance_atb24_conservative['year'].apply(
    lambda y: batt_res_eff_conservative_values[2022] if y < 2022 else batt_res_eff_conservative_values.get(y, None)
)

batt_com_eff_moderate = batt_com_eff[batt_com_eff.iloc[:, 1] == 'Moderate'] # moderate - Commercial and Industrial
batt_com_eff_moderate.columns = batt_com_eff_moderate.columns.astype(int)
batt_com_eff_moderate_values = batt_com_eff_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_tech_performance_atb24_moderate['batt_eff_com'] = batt_tech_performance_atb24_moderate['year'].apply(
    lambda y: batt_com_eff_moderate_values[2022] if y < 2022 else batt_com_eff_moderate_values.get(y, None)
)
batt_tech_performance_atb24_moderate['batt_eff_ind'] = batt_tech_performance_atb24_moderate['year'].apply(
    lambda y: batt_com_eff_moderate_values[2022] if y < 2022 else batt_com_eff_moderate_values.get(y, None)
)

batt_res_eff_moderate = batt_res_eff[batt_res_eff.iloc[:, 1] == 'Moderate'] # moderate - Residential
batt_res_eff_moderate.columns = batt_res_eff_moderate.columns.astype(int)
batt_res_eff_moderate_values = batt_res_eff_moderate.loc[:, 2022:2050].squeeze().to_dict()
batt_tech_performance_atb24_moderate['batt_eff_res'] = batt_tech_performance_atb24_moderate['year'].apply(
    lambda y: batt_res_eff_moderate_values[2022] if y < 2022 else batt_res_eff_moderate_values.get(y, None)
)

batt_com_eff_advanced = batt_com_eff[batt_com_eff.iloc[:, 1] == 'Advanced'] # advanced - Commercial and Industrial
batt_com_eff_advanced.columns = batt_com_eff_advanced.columns.astype(int)
batt_com_eff_advanced_values = batt_com_eff_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_tech_performance_atb24_advanced['batt_eff_com'] = batt_tech_performance_atb24_advanced['year'].apply(
    lambda y: batt_com_eff_advanced_values[2022] if y < 2022 else batt_com_eff_advanced_values.get(y, None)
)
batt_tech_performance_atb24_advanced['batt_eff_ind'] = batt_tech_performance_atb24_advanced['year'].apply(
    lambda y: batt_com_eff_advanced_values[2022] if y < 2022 else batt_com_eff_advanced_values.get(y, None)
)

batt_res_eff_advanced = batt_res_eff[batt_res_eff.iloc[:, 1] == 'Advanced'] # advanced - Residential
batt_res_eff_advanced.columns = batt_res_eff_advanced.columns.astype(int)
batt_res_eff_advanced_values = batt_res_eff_advanced.loc[:, 2022:2050].squeeze().to_dict()
batt_tech_performance_atb24_advanced['batt_eff_res'] = batt_tech_performance_atb24_advanced['year'].apply(
    lambda y: batt_res_eff_advanced_values[2022] if y < 2022 else batt_res_eff_advanced_values.get(y, None)
)

#display(batt_tech_performance_atb24_conservative)
#display(batt_tech_performance_atb24_moderate)
#display(batt_tech_performance_atb24_advanced)

# Save the csv files
output_path = os.path.join(os.getcwd(), '..', 'batt_tech_performance', 'batt_tech_performance_atb24_conservative.csv')
batt_tech_performance_atb24_conservative.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'batt_tech_performance', 'batt_tech_performance_atb24_moderate.csv')
batt_tech_performance_atb24_moderate.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'batt_tech_performance', 'batt_tech_performance_atb24_advanced.csv')
batt_tech_performance_atb24_advanced.to_csv(output_path, index=False)

Successfully read ATB data from: 2024_v3_Workbook.xlsx


## PV + Battery Prices

In [309]:
# Input Data Processing: PV + Battery Prices

# Input Folder: pv_plus_batt_prices
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: 'PV + Storage Cost Scenario'
# Data Source: ATB_Data

#--------------------#

# Import the ATB data
pv_price_atb24_conservative = pd.read_csv('../pv_prices/pv_price_atb24_conservative.csv')
pv_price_atb24_moderate = pd.read_csv('../pv_prices/pv_price_atb24_moderate.csv')
pv_price_atb24_advanced = pd.read_csv('../pv_prices/pv_price_atb24_advanced.csv')

batt_price_atb24_conservative = pd.read_csv('../batt_prices/batt_price_atb24_conservative.csv')
batt_price_atb24_moderate = pd.read_csv('../batt_prices/batt_price_atb24_moderate.csv')
batt_price_atb24_advanced = pd.read_csv('../batt_prices/batt_price_atb24_advanced.csv')

#display(pv_price_atb24_conservative)
#display(batt_price_atb24_conservative)

# Initialize empty data frames for the csv output files (conservative, moderate, advanced)
pv_plus_batt_prices_column_names = [
    'year',
    'tech',
    'system_capex_per_kw_res',
    'system_capex_per_kw_nonres',
    'batt_capex_per_kwh_res',
    'batt_capex_per_kw_res',
    'batt_capex_per_kwh_nonres',
    'batt_capex_per_kw_nonres',
    'linear_constant_res',
    'linear_constant_nonres',
    'batt_om_per_kw_res',
    'batt_om_per_kwh_res',
    'batt_om_per_kw_nonres',
    'batt_om_per_kwh_nonres',
    'batt_replace_frac_kw',
    'batt_replace_frac_kwh'
]
pv_plus_batt_price_atb24_conservative = pd.DataFrame(columns=pv_plus_batt_prices_column_names)
pv_plus_batt_price_atb24_moderate = pd.DataFrame(columns=pv_plus_batt_prices_column_names)
pv_plus_batt_price_atb24_advanced = pd.DataFrame(columns=pv_plus_batt_prices_column_names)

# Populate the data frames
years = list(range(start_year, end_year + 1))
pv_plus_batt_price_atb24_conservative['year'] = years
pv_plus_batt_price_atb24_moderate['year'] = years
pv_plus_batt_price_atb24_advanced['year'] = years
pv_plus_batt_price_atb24_conservative['batt_replace_frac_kw'] = 0.174
pv_plus_batt_price_atb24_moderate['batt_replace_frac_kw'] = 0.174
pv_plus_batt_price_atb24_advanced['batt_replace_frac_kw'] = 0.174
pv_plus_batt_price_atb24_conservative['batt_replace_frac_kwh'] = 1
pv_plus_batt_price_atb24_moderate['batt_replace_frac_kwh'] = 1
pv_plus_batt_price_atb24_advanced['batt_replace_frac_kwh'] = 1
pv_plus_batt_price_atb24_conservative['tech'] = 'pv,batt'
pv_plus_batt_price_atb24_moderate['tech'] = 'pv,batt'
pv_plus_batt_price_atb24_advanced['tech'] = 'pv,batt'

##### PV Data
pv_plus_batt_price_atb24_conservative['system_capex_per_kw_nonres'] = pv_price_atb24_conservative['system_capex_per_kw_com'].values # Nonresidential
pv_plus_batt_price_atb24_moderate['system_capex_per_kw_nonres'] = pv_price_atb24_moderate['system_capex_per_kw_com'].values
pv_plus_batt_price_atb24_advanced['system_capex_per_kw_nonres'] = pv_price_atb24_advanced['system_capex_per_kw_com'].values

pv_plus_batt_price_atb24_conservative['system_capex_per_kw_res'] = pv_price_atb24_conservative['system_capex_per_kw_res'].values # Residential
pv_plus_batt_price_atb24_moderate['system_capex_per_kw_res'] = pv_price_atb24_moderate['system_capex_per_kw_res'].values
pv_plus_batt_price_atb24_advanced['system_capex_per_kw_res'] = pv_price_atb24_advanced['system_capex_per_kw_res'].values

##### Battery Data
pv_plus_batt_price_atb24_conservative['batt_capex_per_kwh_nonres'] = batt_price_atb24_conservative['batt_capex_per_kwh_nonres'].values # Nonresidential
pv_plus_batt_price_atb24_moderate['batt_capex_per_kwh_nonres'] = batt_price_atb24_moderate['batt_capex_per_kwh_nonres'].values
pv_plus_batt_price_atb24_advanced['batt_capex_per_kwh_nonres'] = batt_price_atb24_advanced['batt_capex_per_kwh_nonres'].values

pv_plus_batt_price_atb24_conservative['batt_capex_per_kwh_res'] = batt_price_atb24_conservative['batt_capex_per_kwh_res'].values # Residential
pv_plus_batt_price_atb24_moderate['batt_capex_per_kwh_res'] = batt_price_atb24_moderate['batt_capex_per_kwh_res'].values
pv_plus_batt_price_atb24_advanced['batt_capex_per_kwh_res'] = batt_price_atb24_advanced['batt_capex_per_kwh_res'].values

pv_plus_batt_price_atb24_conservative['batt_capex_per_kw_nonres'] = batt_price_atb24_conservative['batt_capex_per_kw_nonres'].values # Nonresidential
pv_plus_batt_price_atb24_moderate['batt_capex_per_kw_nonres'] = batt_price_atb24_moderate['batt_capex_per_kw_nonres'].values
pv_plus_batt_price_atb24_advanced['batt_capex_per_kw_nonres'] = batt_price_atb24_advanced['batt_capex_per_kw_nonres'].values

pv_plus_batt_price_atb24_conservative['batt_capex_per_kw_res'] = batt_price_atb24_conservative['batt_capex_per_kw_res'].values # Residential
pv_plus_batt_price_atb24_moderate['batt_capex_per_kw_res'] = batt_price_atb24_moderate['batt_capex_per_kw_res'].values
pv_plus_batt_price_atb24_advanced['batt_capex_per_kw_res'] = batt_price_atb24_advanced['batt_capex_per_kw_res'].values

pv_plus_batt_price_atb24_conservative['batt_om_per_kwh_nonres'] = batt_price_atb24_conservative['batt_om_per_kwh_nonres'].values # Nonresidential
pv_plus_batt_price_atb24_moderate['batt_om_per_kwh_nonres'] = batt_price_atb24_moderate['batt_om_per_kwh_nonres'].values
pv_plus_batt_price_atb24_advanced['batt_om_per_kwh_nonres'] = batt_price_atb24_advanced['batt_om_per_kwh_nonres'].values

pv_plus_batt_price_atb24_conservative['batt_om_per_kwh_res'] = batt_price_atb24_conservative['batt_om_per_kwh_res'].values # Residential
pv_plus_batt_price_atb24_moderate['batt_om_per_kwh_res'] = batt_price_atb24_moderate['batt_om_per_kwh_res'].values
pv_plus_batt_price_atb24_advanced['batt_om_per_kwh_res'] = batt_price_atb24_advanced['batt_om_per_kwh_res'].values

pv_plus_batt_price_atb24_conservative['batt_om_per_kw_nonres'] = batt_price_atb24_conservative['batt_om_per_kw_nonres'].values # Nonresidential
pv_plus_batt_price_atb24_moderate['batt_om_per_kw_nonres'] = batt_price_atb24_moderate['batt_om_per_kw_nonres'].values
pv_plus_batt_price_atb24_advanced['batt_om_per_kw_nonres'] = batt_price_atb24_advanced['batt_om_per_kw_nonres'].values

pv_plus_batt_price_atb24_conservative['batt_om_per_kw_res'] = batt_price_atb24_conservative['batt_om_per_kw_res'].values # Residential
pv_plus_batt_price_atb24_moderate['batt_om_per_kw_res'] = batt_price_atb24_moderate['batt_om_per_kw_res'].values
pv_plus_batt_price_atb24_advanced['batt_om_per_kw_res'] = batt_price_atb24_advanced['batt_om_per_kw_res'].values

pv_plus_batt_price_atb24_conservative['linear_constant_nonres'] = batt_price_atb24_conservative['linear_constant_nonres'].values # Nonresidential
pv_plus_batt_price_atb24_moderate['linear_constant_nonres'] = batt_price_atb24_moderate['linear_constant_nonres'].values
pv_plus_batt_price_atb24_advanced['linear_constant_nonres'] = batt_price_atb24_advanced['linear_constant_nonres'].values

pv_plus_batt_price_atb24_conservative['linear_constant_res'] = batt_price_atb24_conservative['linear_constant_res'].values # Residential
pv_plus_batt_price_atb24_moderate['linear_constant_res'] = batt_price_atb24_moderate['linear_constant_res'].values
pv_plus_batt_price_atb24_advanced['linear_constant_res'] = batt_price_atb24_advanced['linear_constant_res'].values

#display(pv_plus_batt_price_atb24_conservative)
#display(pv_plus_batt_price_atb24_moderate)
#display(pv_plus_batt_price_atb24_advanced)

# Save the csv files
output_path = os.path.join(os.getcwd(), '..', 'pv_plus_batt_prices', 'pv_plus_batt_price_atb24_conservative.csv')
pv_plus_batt_price_atb24_conservative.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'pv_plus_batt_prices', 'pv_plus_batt_price_atb24_moderate.csv')
pv_plus_batt_price_atb24_moderate.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'pv_plus_batt_prices', 'pv_plus_batt_price_atb24_advanced.csv')
pv_plus_batt_price_atb24_advanced.to_csv(output_path, index=False)

## Financial Inputs

In [2]:
# Input Data Processing: Financial Inputs

# Input Folder: financing_terms
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: 'Financing Scenario'
# Data Source: ATB_Data

#--------------------#

# Import the ATB data
# Financial Scenario Options: R&D, Market
# Financial Scenario Selected: Market
atb_file_path = os.path.join(atb_data_folder, atb_data_file)
atb_data = pd.ExcelFile(atb_file_path)
print(f'Successfully read ATB data from: {atb_data_file}')

# Assign the individual tabs to separate data frames
pv_dist_com_full = atb_data.parse('Solar - PV Dist. Comm')
pv_dist_res_full = atb_data.parse('Solar - PV Dist. Res')

# Extract the relevant data
pv_dist_com_finance = pv_dist_com_full.iloc[56:87, 10:41] # Commercial
pv_dist_com_finance.columns = pv_dist_com_finance.iloc[0]
pv_dist_com_finance = pv_dist_com_finance[1:]
pv_dist_com_finance.columns = pv_dist_com_finance.columns.astype(float).fillna(0).astype(int).astype(str)
pv_dist_com_finance.columns.values[0] = 0
pv_dist_com_finance.columns.values[1] = 1

pv_dist_res_finance = pv_dist_res_full.iloc[56:88, 10:41] # Residential
pv_dist_res_finance.columns = pv_dist_res_finance.iloc[0]
pv_dist_res_finance = pv_dist_res_finance[1:]
pv_dist_res_finance.columns = pv_dist_res_finance.columns.astype(float).fillna(0).astype(int).astype(str)
pv_dist_res_finance.columns.values[0] = 0
pv_dist_res_finance.columns.values[1] = 1

#display(pv_dist_com_finance)
#display(pv_dist_res_finance)

##### Loan Interest Rate (ATB Assumption: "Calculated Interest Rate Real")
pv_dist_com_loanrate = pv_dist_com_finance.iloc [4:7, :] # Commercial
pv_dist_res_loanrate = pv_dist_res_finance.iloc [4:7, :] # Residential
#display(pv_dist_com_loanrate)
#display(pv_dist_res_loanrate)

##### Real Discount Rate (ATB Assumption: "WACC Real")
pv_dist_com_discountrate = pv_dist_com_finance.iloc [21:24, :] # Commercial
pv_dist_res_discountrate = pv_dist_res_finance.iloc [21:24, :] # Residential
#display(pv_dist_com_discountrate)
#display(pv_dist_res_discountrate)

##### Tax Rate 
pv_dist_com_taxrate = pv_dist_res_finance.iloc [17:18, :] # Commercial
pv_dist_res_taxrate = pv_dist_res_finance.iloc [17:18, :] # Residential
#display(pv_dist_com_taxrate)
#display(pv_dist_res_taxrate)

# Initialize empty data frames for the csv output files (conservative, moderate, advanced)
financing_column_names = [
    'year',
    'economic_lifetime_yrs',
    'loan_term_yrs_res',
    'loan_interest_rate_res',
    'down_payment_fraction_res',
    'real_discount_rate_res',
    'tax_rate_res',
    'loan_term_yrs_nonres',
    'loan_interest_rate_nonres',
    'down_payment_fraction_nonres',
    'real_discount_rate_nonres',
    'tax_rate_nonres'
]
financing_atb24_conservative = pd.DataFrame(columns=financing_column_names)
financing_atb24_moderate = pd.DataFrame(columns=financing_column_names)
financing_atb24_advanced = pd.DataFrame(columns=financing_column_names)

# Populate the data frames
years = list(range(start_year, end_year + 1))
financing_atb24_conservative['year'] = years
financing_atb24_moderate['year'] = years
financing_atb24_advanced['year'] = years

financing_atb24_conservative['economic_lifetime_yrs'] = 30
financing_atb24_moderate['economic_lifetime_yrs'] = 30
financing_atb24_advanced['economic_lifetime_yrs'] = 30

financing_atb24_conservative['loan_term_yrs_res'] = 30
financing_atb24_moderate['loan_term_yrs_res'] = 30
financing_atb24_advanced['loan_term_yrs_res'] = 30

financing_atb24_conservative['down_payment_fraction_res'] = 1
financing_atb24_moderate['down_payment_fraction_res'] = 1
financing_atb24_advanced['down_payment_fraction_res'] = 1

financing_atb24_conservative['loan_term_yrs_nonres'] = 30
financing_atb24_moderate['loan_term_yrs_nonres'] = 30
financing_atb24_advanced['loan_term_yrs_nonres'] = 30

financing_atb24_conservative['down_payment_fraction_nonres'] = 1
financing_atb24_moderate['down_payment_fraction_nonres'] = 1
financing_atb24_advanced['down_payment_fraction_nonres'] = 1

##### Loan Interest Rate (ATB Assumption: "Calculated Interest Rate Real")
pv_dist_com_loanrate_conservative = pv_dist_com_loanrate[pv_dist_com_loanrate.iloc[:, 1] == 'Conservative'] # conservative - Nonresidential
pv_dist_com_loanrate_conservative.columns = pv_dist_com_loanrate_conservative.columns.astype(int)
pv_dist_com_loanrate_conservative_values = pv_dist_com_loanrate_conservative.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_conservative['loan_interest_rate_nonres'] = financing_atb24_conservative['year'].apply(
    lambda y: pv_dist_com_loanrate_conservative_values[2022] if y < 2022 else pv_dist_com_loanrate_conservative_values.get(y, None)
)
pv_dist_res_loanrate_conservative = pv_dist_res_loanrate[pv_dist_res_loanrate.iloc[:, 1] == 'Conservative'] # conservative - Residential
pv_dist_res_loanrate_conservative.columns = pv_dist_res_loanrate_conservative.columns.astype(int)
pv_dist_res_loanrate_conservative_values = pv_dist_res_loanrate_conservative.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_conservative['loan_interest_rate_res'] = financing_atb24_conservative['year'].apply(
    lambda y: pv_dist_res_loanrate_conservative_values[2022] if y < 2022 else pv_dist_res_loanrate_conservative_values.get(y, None)
)
pv_dist_com_loanrate_moderate = pv_dist_com_loanrate[pv_dist_com_loanrate.iloc[:, 1] == 'Moderate'] # moderate - Nonresidential
pv_dist_com_loanrate_moderate.columns = pv_dist_com_loanrate_moderate.columns.astype(int)
pv_dist_com_loanrate_moderate_values = pv_dist_com_loanrate_moderate.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_moderate['loan_interest_rate_nonres'] = financing_atb24_moderate['year'].apply(
    lambda y: pv_dist_com_loanrate_moderate_values[2022] if y < 2022 else pv_dist_com_loanrate_moderate_values.get(y, None)
)
pv_dist_res_loanrate_moderate = pv_dist_res_loanrate[pv_dist_res_loanrate.iloc[:, 1] == 'Moderate'] # moderate - Residential
pv_dist_res_loanrate_moderate.columns = pv_dist_res_loanrate_moderate.columns.astype(int)
pv_dist_res_loanrate_moderate_values = pv_dist_res_loanrate_moderate.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_moderate['loan_interest_rate_res'] = financing_atb24_moderate['year'].apply(
    lambda y: pv_dist_res_loanrate_moderate_values[2022] if y < 2022 else pv_dist_res_loanrate_moderate_values.get(y, None)
)
pv_dist_com_loanrate_advanced = pv_dist_com_loanrate[pv_dist_com_loanrate.iloc[:, 1] == 'Advanced'] # advanced - Nonresidential
pv_dist_com_loanrate_advanced.columns = pv_dist_com_loanrate_advanced.columns.astype(int)
pv_dist_com_loanrate_advanced_values = pv_dist_com_loanrate_advanced.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_advanced['loan_interest_rate_nonres'] = financing_atb24_advanced['year'].apply(
    lambda y: pv_dist_com_loanrate_advanced_values[2022] if y < 2022 else pv_dist_com_loanrate_advanced_values.get(y, None)
)
pv_dist_res_loanrate_advanced = pv_dist_res_loanrate[pv_dist_res_loanrate.iloc[:, 1] == 'Advanced'] # advanced - Residential
pv_dist_res_loanrate_advanced.columns = pv_dist_res_loanrate_advanced.columns.astype(int)
pv_dist_res_loanrate_advanced_values = pv_dist_res_loanrate_advanced.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_advanced['loan_interest_rate_res'] = financing_atb24_advanced['year'].apply(
    lambda y: pv_dist_res_loanrate_advanced_values[2022] if y < 2022 else pv_dist_res_loanrate_advanced_values.get(y, None)
)

##### Real Discount Rate (ATB Assumption: "WACC Real")
pv_dist_com_discountrate_conservative = pv_dist_com_discountrate[pv_dist_com_discountrate.iloc[:, 1] == 'Conservative'] # conservative - Nonresidential
pv_dist_com_discountrate_conservative.columns = pv_dist_com_discountrate_conservative.columns.astype(int)
pv_dist_com_discountrate_conservative_values = pv_dist_com_discountrate_conservative.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_conservative['real_discount_rate_nonres'] = financing_atb24_conservative['year'].apply(
    lambda y: pv_dist_com_discountrate_conservative_values[2022] if y < 2022 else pv_dist_com_discountrate_conservative_values.get(y, None)
)
pv_dist_res_discountrate_conservative = pv_dist_res_discountrate[pv_dist_res_discountrate.iloc[:, 1] == 'Conservative'] # conservative - Residential
pv_dist_res_discountrate_conservative.columns = pv_dist_res_discountrate_conservative.columns.astype(int)
pv_dist_res_discountrate_conservative_values = pv_dist_res_discountrate_conservative.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_conservative['real_discount_rate_res'] = financing_atb24_conservative['year'].apply(
    lambda y: pv_dist_res_discountrate_conservative_values[2022] if y < 2022 else pv_dist_res_discountrate_conservative_values.get(y, None)
)
pv_dist_com_discountrate_moderate = pv_dist_com_discountrate[pv_dist_com_discountrate.iloc[:, 1] == 'Moderate'] # moderate - Nonresidential
pv_dist_com_discountrate_moderate.columns = pv_dist_com_discountrate_moderate.columns.astype(int)
pv_dist_com_discountrate_moderate_values = pv_dist_com_discountrate_moderate.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_moderate['real_discount_rate_nonres'] = financing_atb24_moderate['year'].apply(
    lambda y: pv_dist_com_discountrate_moderate_values[2022] if y < 2022 else pv_dist_com_discountrate_moderate_values.get(y, None)
)
pv_dist_res_discountrate_moderate = pv_dist_res_discountrate[pv_dist_res_discountrate.iloc[:, 1] == 'Moderate'] # moderate - Residential
pv_dist_res_discountrate_moderate.columns = pv_dist_res_discountrate_moderate.columns.astype(int)
pv_dist_res_discountrate_moderate_values = pv_dist_res_discountrate_moderate.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_moderate['real_discount_rate_res'] = financing_atb24_moderate['year'].apply(
    lambda y: pv_dist_res_discountrate_moderate_values[2022] if y < 2022 else pv_dist_res_discountrate_moderate_values.get(y, None)
)
pv_dist_com_discountrate_advanced = pv_dist_com_discountrate[pv_dist_com_discountrate.iloc[:, 1] == 'Advanced'] # advanced - Nonresidential
pv_dist_com_discountrate_advanced.columns = pv_dist_com_discountrate_advanced.columns.astype(int)
pv_dist_com_discountrate_advanced_values = pv_dist_com_discountrate_advanced.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_advanced['real_discount_rate_nonres'] = financing_atb24_advanced['year'].apply(
    lambda y: pv_dist_com_discountrate_advanced_values[2022] if y < 2022 else pv_dist_com_discountrate_advanced_values.get(y, None)
)
pv_dist_res_discountrate_advanced = pv_dist_res_discountrate[pv_dist_res_discountrate.iloc[:, 1] == 'Advanced'] # advanced - Residential
pv_dist_res_discountrate_advanced.columns = pv_dist_res_discountrate_advanced.columns.astype(int)
pv_dist_res_discountrate_advanced_values = pv_dist_res_discountrate_advanced.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_advanced['real_discount_rate_res'] = financing_atb24_advanced['year'].apply(
    lambda y: pv_dist_res_discountrate_advanced_values[2022] if y < 2022 else pv_dist_res_discountrate_advanced_values.get(y, None)
)

##### Tax Rate
pv_dist_com_taxrate_conservative = pv_dist_com_taxrate # conservative - Nonresidential
pv_dist_com_taxrate_conservative.columns = pv_dist_com_taxrate_conservative.columns.astype(int)
pv_dist_com_taxrate_conservative_values = pv_dist_com_taxrate_conservative.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_conservative['tax_rate_nonres'] = financing_atb24_conservative['year'].apply(
    lambda y: pv_dist_com_taxrate_conservative_values[2022] if y < 2022 else pv_dist_com_taxrate_conservative_values.get(y, None)
)
pv_dist_res_taxrate_conservative = pv_dist_res_taxrate # conservative - Residential
pv_dist_res_taxrate_conservative.columns = pv_dist_res_taxrate_conservative.columns.astype(int)
pv_dist_res_taxrate_conservative_values = pv_dist_res_taxrate_conservative.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_conservative['tax_rate_res'] = financing_atb24_conservative['year'].apply(
    lambda y: pv_dist_res_taxrate_conservative_values[2022] if y < 2022 else pv_dist_res_taxrate_conservative_values.get(y, None)
)
pv_dist_com_taxrate_moderate = pv_dist_com_taxrate # moderate - Nonresidential
pv_dist_com_taxrate_moderate.columns = pv_dist_com_taxrate_moderate.columns.astype(int)
pv_dist_com_taxrate_moderate_values = pv_dist_com_taxrate_moderate.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_moderate['tax_rate_nonres'] = financing_atb24_moderate['year'].apply(
    lambda y: pv_dist_com_taxrate_moderate_values[2022] if y < 2022 else pv_dist_com_taxrate_moderate_values.get(y, None)
)
pv_dist_res_taxrate_moderate = pv_dist_res_taxrate # moderate - Residential
pv_dist_res_taxrate_moderate.columns = pv_dist_res_taxrate_moderate.columns.astype(int)
pv_dist_res_taxrate_moderate_values = pv_dist_res_taxrate_moderate.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_moderate['tax_rate_res'] = financing_atb24_moderate['year'].apply(
    lambda y: pv_dist_res_taxrate_moderate_values[2022] if y < 2022 else pv_dist_res_taxrate_moderate_values.get(y, None)
)
pv_dist_com_taxrate_advanced = pv_dist_com_taxrate # advanced - Nonresidential
pv_dist_com_taxrate_advanced.columns = pv_dist_com_taxrate_advanced.columns.astype(int)
pv_dist_com_taxrate_advanced_values = pv_dist_com_taxrate_advanced.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_advanced['tax_rate_nonres'] = financing_atb24_advanced['year'].apply(
    lambda y: pv_dist_com_taxrate_advanced_values[2022] if y < 2022 else pv_dist_com_taxrate_advanced_values.get(y, None)
)
pv_dist_res_taxrate_advanced = pv_dist_res_taxrate # advanced - Residential
pv_dist_res_taxrate_advanced.columns = pv_dist_res_taxrate_advanced.columns.astype(int)
pv_dist_res_taxrate_advanced_values = pv_dist_res_taxrate_advanced.loc[:, 2022:2050].squeeze().to_dict()
financing_atb24_advanced['tax_rate_res'] = financing_atb24_advanced['year'].apply(
    lambda y: pv_dist_res_taxrate_advanced_values[2022] if y < 2022 else pv_dist_res_taxrate_advanced_values.get(y, None)
)

#display(financing_atb24_conservative)
#display(financing_atb24_moderate)
#display(financing_atb24_advanced)

# Save the csv files
output_path = os.path.join(os.getcwd(), '..', 'financing_terms', 'financing_atb24_conservative.csv')
financing_atb24_conservative.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'financing_terms', 'financing_atb24_moderate.csv')
financing_atb24_moderate.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'financing_terms', 'financing_atb24_advanced.csv')
financing_atb24_advanced.to_csv(output_path, index=False)

Successfully read ATB data from: 2024_v3_Workbook.xlsx


## Depreciation Schedules

In [119]:
# Input Data Processing: Depreciation Schedules

# Input Folder: depreciation schedules
# dgen\dgen_os\input_scenarios\input_sheet_final.xlsm: 'Depreciation Scenario'
# Data Source: ATB_Data

#--------------------#

# Import the ATB data
atb_file_path = os.path.join(atb_data_folder, atb_data_file)
atb_data = pd.ExcelFile(atb_file_path)
print(f'Successfully read ATB data from: {atb_data_file}')

# Assign the individual tabs to separate data frames
pv_dist_com_full = atb_data.parse('Solar - PV Dist. Comm')
pv_dist_res_full = atb_data.parse('Solar - PV Dist. Res')

# Extract the relevant data
pv_dist_com_depreciation = pv_dist_com_full.iloc[365:386, 10:41] # Commercial
pv_dist_com_depreciation.columns = pv_dist_com_depreciation.iloc[0]
pv_dist_com_depreciation = pv_dist_com_depreciation[1:]
pv_dist_com_depreciation.columns.values[0] = 0
pv_dist_com_depreciation.columns.values[1] = 1
pv_dist_com_depreciation.columns = pv_dist_com_depreciation.columns.astype(float).fillna(0).astype(int).astype(str)

pv_dist_res_depreciation = pv_dist_res_full.iloc[365:386, 10:41] # Residential
pv_dist_res_depreciation.columns = pv_dist_res_depreciation.iloc[0]
pv_dist_res_depreciation = pv_dist_res_depreciation[1:]
pv_dist_res_depreciation.columns.values[0] = 0
pv_dist_res_depreciation.columns.values[1] = 1
pv_dist_res_depreciation.columns = pv_dist_res_depreciation.columns.astype(float).fillna(0).astype(int).astype(str)

##### Conservative
pv_dist_com_depreciation_conservative = pv_dist_com_depreciation.iloc[14:20, :] # Commercial
pv_dist_com_depreciation_conservative = pv_dist_com_depreciation_conservative.T
new_header = pv_dist_com_depreciation_conservative.iloc[0]
pv_dist_com_depreciation_conservative = pv_dist_com_depreciation_conservative[2:]
pv_dist_com_depreciation_conservative.columns = new_header

pv_dist_res_depreciation_conservative = pv_dist_res_depreciation.iloc[14:20, :] # Residential
pv_dist_res_depreciation_conservative = pv_dist_res_depreciation_conservative.T
new_header = pv_dist_res_depreciation_conservative.iloc[0]
pv_dist_res_depreciation_conservative = pv_dist_res_depreciation_conservative[2:]
pv_dist_res_depreciation_conservative.columns = new_header

##### Moderate
pv_dist_com_depreciation_moderate = pv_dist_com_depreciation.iloc[7:13, :] # Commercial
pv_dist_com_depreciation_moderate = pv_dist_com_depreciation_moderate.T
new_header = pv_dist_com_depreciation_moderate.iloc[0]
pv_dist_com_depreciation_moderate = pv_dist_com_depreciation_moderate[2:]
pv_dist_com_depreciation_moderate.columns = new_header

pv_dist_res_depreciation_moderate = pv_dist_res_depreciation.iloc[7:13, :] # Residential
pv_dist_res_depreciation_moderate = pv_dist_res_depreciation_moderate.T
new_header = pv_dist_res_depreciation_moderate.iloc[0]
pv_dist_res_depreciation_moderate = pv_dist_res_depreciation_moderate[2:]
pv_dist_res_depreciation_moderate.columns = new_header

##### Advanced
pv_dist_com_depreciation_advanced = pv_dist_com_depreciation.iloc[0:6, :] # Commercial
pv_dist_com_depreciation_advanced = pv_dist_com_depreciation_advanced.T
new_header = pv_dist_com_depreciation_advanced.iloc[0]
pv_dist_com_depreciation_advanced = pv_dist_com_depreciation_advanced[2:]
pv_dist_com_depreciation_advanced.columns = new_header

pv_dist_res_depreciation_advanced = pv_dist_res_depreciation.iloc[0:6, :] # Residential
pv_dist_res_depreciation_advanced = pv_dist_res_depreciation_advanced.T
new_header = pv_dist_res_depreciation_advanced.iloc[0]
pv_dist_res_depreciation_advanced = pv_dist_res_depreciation_advanced[2:]
pv_dist_res_depreciation_advanced.columns = new_header

#display(pv_dist_com_depreciation)
#display(pv_dist_com_depreciation_conservative)
#display(pv_dist_com_depreciation_moderate)
#display(pv_dist_com_depreciation_advanced)
#display(pv_dist_res_depreciation)
#display(pv_dist_res_depreciation_conservative)
#display(pv_dist_res_depreciation_moderate)
#display(pv_dist_res_depreciation_advanced)

# Initialize empty data frames for the csv output files (conservative, moderate, advanced)
depreciation_column_names = [
    'year',
    'sector_abbr',
    1,
    2,
    3,
    4,
    5,
    6,
]
depreciation_atb24_conservative = pd.DataFrame(columns=depreciation_column_names)
depreciation_atb24_moderate = pd.DataFrame(columns=depreciation_column_names)
depreciation_atb24_advanced = pd.DataFrame(columns=depreciation_column_names)

# Populate the data frames
years = list(range(start_year, end_year + 1))
depreciation_atb24_conservative['year'] = years*3
depreciation_atb24_moderate['year'] = years*3
depreciation_atb24_advanced['year'] = years*3

num_years = end_year - start_year + 1
com_entries = ['com'] * num_years
ind_entries = ['ind'] * num_years
res_entries = ['res'] * num_years
sector_abbr_list = com_entries + ind_entries + res_entries
depreciation_atb24_conservative['sector_abbr'] = sector_abbr_list
depreciation_atb24_moderate['sector_abbr'] = sector_abbr_list
depreciation_atb24_advanced['sector_abbr'] = sector_abbr_list

##### Conservative
pv_dist_com_depreciation_conservative.index = pv_dist_com_depreciation_conservative.index.astype(int)
pv_dist_res_depreciation_conservative.index = pv_dist_res_depreciation_conservative.index.astype(int)
for col in [1, 2, 3, 4, 5, 6]:
    # conservative - Commercial
    com_mask_2014_2021 = (depreciation_atb24_conservative['sector_abbr'] == 'com') & (depreciation_atb24_conservative['year'] >= 2014) & (depreciation_atb24_conservative['year'] <= 2021)
    com_mask_2022_2050 = (depreciation_atb24_conservative['sector_abbr'] == 'com') & (depreciation_atb24_conservative['year'] >= 2022) & (depreciation_atb24_conservative['year'] <= 2050)
    depreciation_atb24_conservative.loc[com_mask_2014_2021, col] = pv_dist_com_depreciation_conservative.loc[2022, col]
    depreciation_atb24_conservative.loc[com_mask_2022_2050, col] = pv_dist_com_depreciation_conservative.loc[depreciation_atb24_conservative.loc[com_mask_2022_2050, 'year'], col].astype(depreciation_atb24_conservative[col].dtype).values
    # conservative - Industrial
    ind_mask_2014_2021 = (depreciation_atb24_conservative['sector_abbr'] == 'ind') & (depreciation_atb24_conservative['year'] >= 2014) & (depreciation_atb24_conservative['year'] <= 2021)
    ind_mask_2022_2050 = (depreciation_atb24_conservative['sector_abbr'] == 'ind') & (depreciation_atb24_conservative['year'] >= 2022) & (depreciation_atb24_conservative['year'] <= 2050)
    depreciation_atb24_conservative.loc[ind_mask_2014_2021, col] = pv_dist_com_depreciation_conservative.loc[2022, col]
    depreciation_atb24_conservative.loc[ind_mask_2022_2050, col] = pv_dist_com_depreciation_conservative.loc[depreciation_atb24_conservative.loc[ind_mask_2022_2050, 'year'], col].astype(depreciation_atb24_conservative[col].dtype).values
    # conservative - Residential
    res_mask_2014_2021 = (depreciation_atb24_conservative['sector_abbr'] == 'res') & (depreciation_atb24_conservative['year'] >= 2014) & (depreciation_atb24_conservative['year'] <= 2021)
    res_mask_2022_2050 = (depreciation_atb24_conservative['sector_abbr'] == 'res') & (depreciation_atb24_conservative['year'] >= 2022) & (depreciation_atb24_conservative['year'] <= 2050)
    depreciation_atb24_conservative.loc[res_mask_2014_2021, col] = pv_dist_res_depreciation_conservative.loc[2022, col]
    depreciation_atb24_conservative.loc[res_mask_2022_2050, col] = pv_dist_res_depreciation_conservative.loc[depreciation_atb24_conservative.loc[res_mask_2022_2050, 'year'], col].astype(depreciation_atb24_conservative[col].dtype).values

##### Moderate
pv_dist_com_depreciation_moderate.index = pv_dist_com_depreciation_moderate.index.astype(int)
pv_dist_res_depreciation_moderate.index = pv_dist_res_depreciation_moderate.index.astype(int)
for col in [1, 2, 3, 4, 5, 6]:
    # moderate - Commercial
    com_mask_2014_2021 = (depreciation_atb24_moderate['sector_abbr'] == 'com') & (depreciation_atb24_moderate['year'] >= 2014) & (depreciation_atb24_moderate['year'] <= 2021)
    com_mask_2022_2050 = (depreciation_atb24_moderate['sector_abbr'] == 'com') & (depreciation_atb24_moderate['year'] >= 2022) & (depreciation_atb24_moderate['year'] <= 2050)
    depreciation_atb24_moderate.loc[com_mask_2014_2021, col] = pv_dist_com_depreciation_moderate.loc[2022, col]
    depreciation_atb24_moderate.loc[com_mask_2022_2050, col] = pv_dist_com_depreciation_moderate.loc[depreciation_atb24_moderate.loc[com_mask_2022_2050, 'year'], col].astype(depreciation_atb24_moderate[col].dtype).values
    # moderate - Industrial
    ind_mask_2014_2021 = (depreciation_atb24_moderate['sector_abbr'] == 'ind') & (depreciation_atb24_moderate['year'] >= 2014) & (depreciation_atb24_moderate['year'] <= 2021)
    ind_mask_2022_2050 = (depreciation_atb24_moderate['sector_abbr'] == 'ind') & (depreciation_atb24_moderate['year'] >= 2022) & (depreciation_atb24_moderate['year'] <= 2050)
    depreciation_atb24_moderate.loc[ind_mask_2014_2021, col] = pv_dist_com_depreciation_moderate.loc[2022, col]
    depreciation_atb24_moderate.loc[ind_mask_2022_2050, col] = pv_dist_com_depreciation_moderate.loc[depreciation_atb24_moderate.loc[ind_mask_2022_2050, 'year'], col].astype(depreciation_atb24_moderate[col].dtype).values
    # moderate - Residential
    res_mask_2014_2021 = (depreciation_atb24_moderate['sector_abbr'] == 'res') & (depreciation_atb24_moderate['year'] >= 2014) & (depreciation_atb24_moderate['year'] <= 2021)
    res_mask_2022_2050 = (depreciation_atb24_moderate['sector_abbr'] == 'res') & (depreciation_atb24_moderate['year'] >= 2022) & (depreciation_atb24_moderate['year'] <= 2050)
    depreciation_atb24_moderate.loc[res_mask_2014_2021, col] = pv_dist_res_depreciation_moderate.loc[2022, col]
    depreciation_atb24_moderate.loc[res_mask_2022_2050, col] = pv_dist_res_depreciation_moderate.loc[depreciation_atb24_moderate.loc[res_mask_2022_2050, 'year'], col].astype(depreciation_atb24_moderate[col].dtype).values

##### Advanced
pv_dist_com_depreciation_advanced.index = pv_dist_com_depreciation_advanced.index.astype(int)
pv_dist_res_depreciation_advanced.index = pv_dist_res_depreciation_advanced.index.astype(int)
for col in [1, 2, 3, 4, 5, 6]:
    # advanced - Commercial
    com_mask_2014_2021 = (depreciation_atb24_advanced['sector_abbr'] == 'com') & (depreciation_atb24_advanced['year'] >= 2014) & (depreciation_atb24_advanced['year'] <= 2021)
    com_mask_2022_2050 = (depreciation_atb24_advanced['sector_abbr'] == 'com') & (depreciation_atb24_advanced['year'] >= 2022) & (depreciation_atb24_advanced['year'] <= 2050)
    depreciation_atb24_advanced.loc[com_mask_2014_2021, col] = pv_dist_com_depreciation_advanced.loc[2022, col]
    depreciation_atb24_advanced.loc[com_mask_2022_2050, col] = pv_dist_com_depreciation_advanced.loc[depreciation_atb24_advanced.loc[com_mask_2022_2050, 'year'], col].astype(depreciation_atb24_advanced[col].dtype).values
    # advanced - Industrial
    ind_mask_2014_2021 = (depreciation_atb24_advanced['sector_abbr'] == 'ind') & (depreciation_atb24_advanced['year'] >= 2014) & (depreciation_atb24_advanced['year'] <= 2021)
    ind_mask_2022_2050 = (depreciation_atb24_advanced['sector_abbr'] == 'ind') & (depreciation_atb24_advanced['year'] >= 2022) & (depreciation_atb24_advanced['year'] <= 2050)
    depreciation_atb24_advanced.loc[ind_mask_2014_2021, col] = pv_dist_com_depreciation_advanced.loc[2022, col]
    depreciation_atb24_advanced.loc[ind_mask_2022_2050, col] = pv_dist_com_depreciation_advanced.loc[depreciation_atb24_advanced.loc[ind_mask_2022_2050, 'year'], col].astype(depreciation_atb24_advanced[col].dtype).values
    # advanced - Residential
    res_mask_2014_2021 = (depreciation_atb24_advanced['sector_abbr'] == 'res') & (depreciation_atb24_advanced['year'] >= 2014) & (depreciation_atb24_advanced['year'] <= 2021)
    res_mask_2022_2050 = (depreciation_atb24_advanced['sector_abbr'] == 'res') & (depreciation_atb24_advanced['year'] >= 2022) & (depreciation_atb24_advanced['year'] <= 2050)
    depreciation_atb24_advanced.loc[res_mask_2014_2021, col] = pv_dist_res_depreciation_advanced.loc[2022, col]
    depreciation_atb24_advanced.loc[res_mask_2022_2050, col] = pv_dist_res_depreciation_advanced.loc[depreciation_atb24_advanced.loc[res_mask_2022_2050, 'year'], col].astype(depreciation_atb24_advanced[col].dtype).values

#display(depreciation_atb24_conservative)
#display(depreciation_atb24_moderate)
#display(depreciation_atb24_advanced)

# Save the csv files
output_path = os.path.join(os.getcwd(), '..', 'depreciation_schedules', 'depreciation_atb24_conservative.csv')
depreciation_atb24_conservative.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'depreciation_schedules', 'depreciation_atb24_moderate.csv')
depreciation_atb24_moderate.to_csv(output_path, index=False)
output_path = os.path.join(os.getcwd(), '..', 'depreciation_schedules', 'depreciation_atb24_advanced.csv')
depreciation_atb24_advanced.to_csv(output_path, index=False)

Successfully read ATB data from: 2024_v3_Workbook.xlsx
